# results
---
### purpose 
this notebook is designed to walk the reader through the results of an analysis of the run-time comparison between python and c++. if successful, it will explain what was tested, how the data was collected, and present the results in a clear way. all the code used to collect the data is located in this github repo, so the reader can recreate the results. 

---
### what was tested
the motivation for this project came when my boss asked me to write a c++ and python version of a QT gui app and compare runtime of different operations. that led to interest in non-graphical speed tests, so the tests are broken down into graphical and non-graphical. below are the four tests that were run.
>__non-graphical__ 
* random number generation
* prime factorization

>__graphical__
* list population
* computation and list population

each test will be explained and the data will be presented in the cells below. enjoy.

## random number generation
data collection files: [python](link) [cpp](link)

--- 
>__method__: 
a list containing the numbers 1 to 500,001 and incrementing by 500 was created, so there were 1000 values. this list served as the size of the rng lists to be created. both programs took this list and generated a new list of random numbers in the range [2,1000000000) of size of the current index of the original list (so first a list of size 1, then of 501 and so on). the rng was of such a large range because larger numbers take longer to factorize so it lended itself well to the next test.

